# Examples of Common Queries

https://dev.mysql.com/doc/refman/8.0/en/examples.html

## 0: Create database and add content

In [27]:
%CREATE common_query_example.db common_query_example_db

In [63]:
DROP TABLE IF EXISTS shop;

In [64]:
CREATE TABLE shop (
    article INT UNSIGNED  DEFAULT '0000' NOT NULL,
    dealer  CHAR(20)      DEFAULT ''     NOT NULL,
    price   DECIMAL(16,2) DEFAULT '0.00' NOT NULL,
    PRIMARY KEY(article, dealer));

In [65]:
INSERT INTO shop VALUES
    (1,'A',3.99),(1,'B',3.99),(2,'A',10.99),(3,'B',1.45),
    (3,'C',1.69),(3,'D',1.25),(4,'D',19.95),(1,'D',19.95);

In [66]:
SELECT * FROM shop ORDER BY dealer, price

article,dealer,price
1,A,3.99
2,A,10.99
3,B,1.45
1,B,3.99
3,C,1.69
3,D,1.25
4,D,19.95
1,D,19.95


## Rows Holding the Group-wise Maximum of a Certain Column
Task: For each dealer, find the max price

In [71]:
SELECT article, dealer, price
FROM   shop s1
WHERE  price=(SELECT MAX(s2.price)
              FROM shop s2
              WHERE s1.dealer = s2.dealer)
ORDER BY article

article,dealer,price
1,B,3.99
1,D,19.95
2,A,10.99
3,C,1.69
4,D,19.95


In [76]:
SELECT * --s1.article, s1.dealer, s1.price
FROM shop s1
JOIN (
  SELECT dealer, MAX(price) AS price
  FROM shop
  GROUP BY dealer) AS s2
  ON s1.dealer = s2.dealer AND s1.price = s2.price
--ORDER BY article;

article,dealer,price,dealer,price
1,B,3.99,B,3.99
2,A,10.99,A,10.99
3,C,1.69,C,1.69
4,D,19.95,D,19.95
1,D,19.95,D,19.95


In [72]:
SELECT article, MAX(price) AS price
  FROM shop
  GROUP BY article

article,price
1,19.95
2,10.99
3,1.69
4,19.95


In [61]:
SELECT s1.article, s1.dealer, s1.price
FROM shop s1
LEFT JOIN shop s2 ON s1.dealer = s2.dealer AND s1.price < s2.price
WHERE s2.article IS NULL
ORDER BY s1.article;

article,dealer,price
1,B,3.99
2,A,10.99
3,C,1.69
4,D,19.95


In [60]:
WITH s1 AS (
   SELECT article, dealer, price,
          RANK() OVER (PARTITION BY dealer
                           ORDER BY price DESC
                      ) AS `Rank`
     FROM shop
)
SELECT article, dealer, price, Rank
  FROM s1
--  WHERE `Rank` = 1
ORDER BY article;

article,dealer,price,Rank
1,A,3.99,2
1,B,3.99,1
2,A,10.99,1
3,B,1.45,2
3,C,1.69,1
3,D,1.25,2
4,D,19.95,1
